Import package
取得CCTV的影片

In [1]:
# -*- coding: utf-8 -*-
import requests
import os
import eventlet
import time
from datetime import datetime
from datetime import timedelta
import threading

Basic setting
D/CCTV檔案要先建好

In [2]:
eventlet.monkey_patch()
DBType = 'cctv_value'
path = 'D:/cctv/'
amount = 512
os.chdir(path)

Define retrieve function for Thread to implement
request URL，因為流量限制，抓取至多20s，Timeout就跳出

In [ ]:
def retrieve(cctv_Id, URL):
    fd = open(cctv_Id + '/' + dstr + '_' + HMstr, 'wb')
    try:
        with eventlet.Timeout(20):
            r = requests.get(URL, stream=True)
            for chunk in r.iter_content(amount):
                fd.write(chunk)
    except:
        pass
    finally:
        fd.close()

14隻CCTV名稱與對應網址

In [ ]:
cctvs = ['nfbCCTV-N5-N-16.915-M', 'nfbCCTV-N5-N-18.308-M', 'nfbCCTV-N5-N-19.707-M', 'nfbCCTV-N5-N-21.056-M',
         'nfbCCTV-N5-N-22.514-M', 'nfbCCTV-N5-N-23.921-M', 'nfbCCTV-N5-N-25.309-M',
         'nfbCCTV-N5-S-16.892-M', 'nfbCCTV-N5-S-18.339-M', 'nfbCCTV-N5-S-19.7-M', 'nfbCCTV-N5-S-21.048-M',
         'nfbCCTV-N5-S-22.493-M', 'nfbCCTV-N5-S-23.896-M', 'nfbCCTV-N5-S-25.298-M']

urls = ['http://202.39.180.202:8081/mjpeg/X01001510210401', 'http://202.39.180.202:8081/mjpeg/X01001510210601',
        'http://202.39.180.202:8081/mjpeg/X01001510211101', 'http://202.39.180.202:8081/mjpeg/X01001510211301',
        'http://202.39.180.202:8081/mjpeg/X01001510211501', 'http://202.39.180.202:8081/mjpeg/X01001510212001',
        'http://202.39.180.202:8081/mjpeg/X01001510212201',
        'http://202.39.180.202:8081/mjpeg/X01001510210101', 'http://202.39.180.202:8081/mjpeg/X01001510210301',
        'http://202.39.180.202:8081/mjpeg/X01001510210701', 'http://202.39.180.202:8081/mjpeg/X01001510210801',
        'http://202.39.180.202:8081/mjpeg/X01001510211001', 'http://202.39.180.202:8081/mjpeg/X01001510211601',
        'http://202.39.180.202:8081/mjpeg/X01001510211701']

設定第一次抓取的時間
跑 num_of_days 天
(datetime.now() + timedelta(seconds=(20 - datetime.now().second)) 結果一定是 xx:xx:20 (設定為最近時間的20秒)

In [ ]:
if datetime.now().second < 20:
    time0 = (datetime.now() + timedelta(seconds=(20 - datetime.now().second))).replace(microsecond=0)
else:
    time0 = (datetime.now() + timedelta(seconds=(80 - datetime.now().second))).replace(microsecond=0)
time1 = time0
num_of_days = 30

for Id in cctvs:
    if not os.path.exists(Id):
        os.makedirs(Id)

電腦秒數為20時抓一次CCTV
datetime.now()為當前電腦時間
(datetime.now() + timedelta(seconds=(20 - datetime.now().second)) 結果一定是 xx:xx:20 (設定為最近時間的20秒)
因為有多個cctv要在同時間去索取影片，因此建立thread來索取

In [ ]:
while (time1 - time0).total_seconds() / 3600 / 24 <= num_of_days:
    if datetime.now().replace(microsecond=0) == time1:
        try:
            dstr = time1.strftime("%Y%m%d")
            HMstr = time1.strftime("%H%M%S")
            print(datetime.now().strftime("%Y%m%d %H%M%S"))
            threads = []
            for i in range(len(cctvs)):
                threads.append(threading.Thread(target=retrieve, args=(cctvs[i], urls[i])))
                threads[i].start()
            for i in range(len(cctvs)):
                threads[i].join()
            time1 = time1 + timedelta(minutes=1)
        except:
            time.sleep(10)
            if datetime.now().second < 20:
                time1 = (datetime.now() + timedelta(seconds=(20 - datetime.now().second))).replace(microsecond=0)
            else:
                time1 = (datetime.now() + timedelta(seconds=(80 - datetime.now().second))).replace(microsecond=0)

